In [1]:
import config.load_data as load_data
from models import model

from tqdm import tqdm
import os.path
import sys
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_regression
from sklearn.model_selection import train_test_split

from ray import tune
from ray.air.integrations.mlflow import MLflowLoggerCallback
from ray.tune.schedulers import ASHAScheduler

import mlflow
from mlflow.tracking import MlflowClient

In [2]:
client = MlflowClient()
cwd = os.getcwd()
exp_base_name = "Models_with_MI"

created = 0
for i in range(100):
    try:
        exp_name = exp_base_name+"_{}".format(i)
        experiment_id = client.create_experiment(exp_name)
        created=1
        break
    except (TypeError, mlflow.exceptions.MlflowException):
        continue

if not created:
    print("ERROR: Try new experiment name.")
    sys.exit(1)

weights_root = "./model_weights/"
weights_dir = weights_root+exp_name+'/'
os.mkdir(weights_dir)

In [3]:
def fit(net, loss_function, optimizer, data_loader, num_epochs, mode, use_amp=False):
	history = {"train": {"loss": [], "mae": []}, "val": {"loss": [], "mae": []}}
	scaler = torch.cuda.amp.GradScaler(enabled=use_amp) # Mixed-precision support for compatible GPUs
	print("\nTraining the model:")
	for epoch in range(num_epochs):
		print("\nEpoch", epoch+1)
		if epoch < num_epochs - 1:
			keys = ["train", "val"]
		else:
			keys = ["train", "val", "test"]
		for key in keys:
			dataset_size = 0
			dataset_loss = 0.0
			if key == "train":
				net.train()
			else:
				net.eval()
			for X_batch, y_batch in tqdm(data_loader[key]):
				X_batch, y_batch = X_batch.to(mode["device"]), y_batch.to(mode["device"])
				with torch.set_grad_enabled(mode=(key=="train")): # Autograd activated only during training
					with torch.cuda.amp.autocast(enabled=use_amp): # Mixed-precision support for compatible GPUs
						batch_output = net(X_batch.float())
						batch_loss = loss_function(batch_output, y_batch.unsqueeze(1))
					if key == "train":
						scaler.scale(batch_loss).backward()
						scaler.step(optimizer) 	
						scaler.update()
						optimizer.zero_grad()
				dataset_size += y_batch.shape[0]
				dataset_loss += y_batch.shape[0] * batch_loss.item()
			dataset_loss /= dataset_size
			if key in ["train", "val"]:
				history[key]["loss"].append(dataset_loss)
				if key == "train":
					tune.report(train_loss=dataset_loss)
				else:
					tune.report(val_loss=dataset_loss)
			else:
				print("\nEvaluating the model:")
			print(key, "loss:", dataset_loss)
			tune.report(test_loss=dataset_loss)
	return net

In [4]:
new_df = pd.read_csv('./data/clean_data/univariate/Q_Kalltveit/72_lags_Q_Kalltveit.csv', index_col="Datetime")

def get_data(mi):
    X = new_df.copy()
    y = X.pop("Q_Kalltveit")
    
    if mi:
        # Label encoding for categoricals
        for colname in X.select_dtypes("object"):
            X[colname], _ = X[colname].factorize()
        discrete_features = X.dtypes == int
        mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
        mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
        mi_scores = mi_scores.sort_values(ascending=False)
        selected_dimention = mi_scores[mi_scores.values >= mi]
        X = X[selected_dimention.index]
        print(selected_dimention.index)
    
    X_train, X_temp, y_train, y_temp = train_test_split(X, y, train_size=.7, shuffle=False)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=.3, shuffle=False)

    X_train, y_train = torch.tensor(X_train.values.astype(np.float32)), torch.tensor(y_train.values.astype(np.float32))
    X_val, y_val = torch.tensor(X_val.values.astype(np.float32)), torch.tensor(y_val.values.astype(np.float32))
    X_test, y_test =  torch.tensor(X_test.values.astype(np.float32)), torch.tensor(y_test.values.astype(np.float32))

    train_, val_, test_ = TensorDataset(X_train, y_train), TensorDataset(X_val, y_val), TensorDataset(X_test, y_test)
    return train_, val_, test_

In [5]:
def train_model(config):
    use_GPU = torch.cuda.is_available()
    if use_GPU:
        mode = {"name": "cuda", "device": torch.device("cuda")}
    else:
        mode = {"name": "cpu", "device": torch.device("cpu")}

    num_epochs = 10
    batch_size = 128*4 #config['batch_size']
    
    lr = 1e-4 # config['lr']
    
    mi = config['mi']
    sequence_length = 72

    train_, val_, test_ = get_data(mi) # Get data with MI

    in_dim = len(train_[0][0])
    lstm_in_dim = int(in_dim/sequence_length)
    lstm_hidden_dim = 64 #config['hidden_dim']
    out_dim = 1

    train_dataloader = torch.utils.data.DataLoader(train_,
                                           batch_size = batch_size,
                                           shuffle = True)
    val_dataloader = torch.utils.data.DataLoader(val_,
                                            batch_size = batch_size,
                                            shuffle = False)
    test_dataloader = torch.utils.data.DataLoader(test_,
                                            batch_size = batch_size,
                                            shuffle = False)

    data_loader = {
    "train": train_dataloader,
    "val": val_dataloader,
    "test": test_dataloader,
    }

    if config['arch'] == "FCN":
        net = model.FCN(in_dim,
                        sequence_length,
                        lstm_in_dim,
                        lstm_hidden_dim,
                        out_dim,
                        mode,)
    elif config['arch'] == "LSTM":
        net = model.LSTM(in_dim,
                        sequence_length,
                        lstm_in_dim,
                        lstm_hidden_dim,
                        out_dim,
                        mode,)
    elif config['arch'] == "TA_LSTM":
        net = model.TA_LSTM(in_dim,
                        sequence_length,
                        lstm_in_dim,
                        lstm_hidden_dim,
                        out_dim,
                        mode,)
    
    net.to(mode["device"])

    loss_function = nn.MSELoss().to(mode["device"])
    optimizer = optim.Adam(net.parameters(), lr=lr)
                                           
    best_trained_model = fit(net, loss_function, optimizer, data_loader, num_epochs, mode)
    out_name = ""
    for k, v in config.items():
        if not k in ['weights_dir', 'cwd']:
            out_name += '{}-{}_'.format(k, v)
    torch.save(best_trained_model.state_dict(), os.path.join(config['cwd'], config['weights_dir'], out_name[:-1] + '.pth'))


In [6]:
config = {
    "mlflow_experiment_id": experiment_id,
    "weights_dir": weights_dir,
    "cwd": cwd,
    #"lr": tune.loguniform(1e-4, 1e-1),
    #"batch_size": tune.choice([128*2, 128*3, 128*4]),
    "mi": tune.grid_search([0, 1, 2]),
    #"in_dim": tune.grid_search([24, 24*2, 24*3]),
    "arch": tune.grid_search(["FCN", "LSTM", "TA_LSTM"]),
    #"hidden_dim": tune.choice([32, 64, 128])
}
"""scheduler = ASHAScheduler(
        metric='val_loss',
        mode="min",
        max_t=100,
        grace_period=1,
        reduction_factor=2,
)"""

analysis = tune.run(
    train_model,
    config=config,
    resources_per_trial={"cpu": 12, "gpu": 1},
    num_samples=2,
    # scheduler=scheduler,
    callbacks=[MLflowLoggerCallback(experiment_name=exp_name)],
)